# 5. Variational Eigensolver (VQE) Algorithm

## Installation

In [ ]:
# Installation code
!pip install numpy
!pip install qiskit
!pip install 'qiskit[visualization]'
!pip install qiskit-nature
!pip install pyscf
!pip install qutip
!pip install ase
!pip install pyqmc --upgrade
!pip install h5py
!pip install scipy


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit.visualization import array_to_latex, plot_bloch_vector, plot_bloch_multivector, plot_state_qsphere, plot_state_city
from qiskit import QuantumRegister, ClassicalRegister,QuantumCircuit, transpile
from qiskit import execute, Aer
import qiskit.quantum_info as qi
from qiskit.extensions import Initialize
#from qiskit.providers.aer import extensions  # import aersnapshot instructions
from qiskit import Aer
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import QubitConverter
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer
from qiskit_nature import settings
from qiskit.providers.aer import StatevectorSimulator
from qiskit import Aer
from qiskit.primitives import Sampler
from qiskit.algorithms import HamiltonianPhaseEstimation, PhaseEstimation
from qiskit.primitives import Estimator
from qiskit.algorithms.optimizers import SLSQP, SPSA, QNSPSA
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit.circuit.library import TwoLocal
from functools import partial as apply_variation

from pyscf import gto, scf
import pyqmc.api as pyq
import h5py
from ase import Atoms
from ase.build import molecule
from ase.visualize import view
import cmath
import math
import scipy.stats as stats
import qutip
import time, datetime
import pandas as pd
import os.path

## 5.1. Variational method

### 5.1.1. The Rayleigh-Ritz variational theorem

Rayleigh-Ritz variational theorem이 말하고자 하는것은 어떤 계의 헤밀토니안 $\hat H$ 의 어떤 임의의 파동함수로 표현되는 어떤상태에 대한 기댓값은 언제나 바닥상태 에너지 $(E_0)$보다는 크다는것을 의미한다. 즉, 이를 수식으로 표현하면 아래와 같다. 
$$E_\Psi = \langle \Psi \vert E \vert \Psi \rangle = \frac{\int \Psi^\dagger \hat H \Psi d\tau}{\int \Psi^\dagger  \Psi d\tau} \geq E_0$$

여기서 적분변수 $\tau$ 는 일반적인 시간, 공간 혹은 스핀 등의 변수를 의미한다. 여기까지의 표현은 아직 어떤 화학적인 제한은 없다. 


위의 식을 증명해보자. 
헤밀토니안에 대해 Completeness Relation 을 사용할것이다. 즉, Hermitian 연산자에대해 언제나 orthonormal 한 basis를 가진다는것을 이용해 전개해볼것이다. 즉,
$$\hat H \vert e_i \rangle = E_i \vert e_i \rangle$$
로부터 아래의 관계식 (Completeness Relation)을 얻을 수 있다. 
$$\hat H = \sum_{i=0}^K E_i \vert e_i \rangle \langle e_i \vert  $$

이 헤밀토니안의 orthonormal 한 basis ($e_i$) 는 Complete 하므로 

e_i 를 통해 어떤 임의의 상태를 표현 할 수 있다. 
$$\vert \Psi_{arbitrary} \rangle = \sum_{k=0}^K c_k \vert e_k \rangle $$
여기서 $c_k = \vert e_k \vert \Psi \rangle$ 이고, 정규화 조건을 만족한다. $\langle \Psi \vert \Psi \rangle = \sum_{k=0}^K {\vert c_k \vert}^2 =1$

이렇게 정의를 해두고 앞의 우리가 보고자했던 헤밀토니안의 기댓값 식 ($\langle \Psi \vert E \vert \Psi \rangle$)에 대입해보자

$$\langle \Psi \vert \hat H \vert \Psi \rangle = \sum_{k=0}^K \langle {c_k}^* e_k \vert \hat H \vert c_k e_k \rangle = \sum_{k=0}^K {c_k}^* c_k \langle  e_k \vert \hat H \vert  e_k \rangle =\sum_{k=0}^K {c_k}^* c_k E_k \langle  e_k \vert e_k \rangle =  \sum_{k=0}^K  E_k {|c_k|}^2 \geq E_0$$

거의다 왔다. $c_k $ 와 $\vert e_k \rangle$ 과, k가 이제 무슨 의미인지를 알아보자. 

$e_k$ 는 Hamiltonian $\hat H $ 의 basis로 정의하였다. 그리고 k번째 basis 와 hamiltonian의 내적값은 k번째 에너지이다. (낮은순서대로)

그래서 $c_0$ 가 1이고, 나머지 $c_k (k \neq 0)$ 가 0일때, 우리가 보고자하는 상태의 헤밀토니안의 기댓값은 바닥상태 에너지 $E_0$이고, 이 이외의 경우에는 무조건 $E_0$ 보다는 큰 에너지를 가지게된다. 즉 아래의 식으로 표현 할 수 있다. 

$$E_\Psi = \frac{\langle \Psi \vert \hat H \vert \Psi \rangle} {\langle \Psi \vert \Psi \rangle} \geq \frac{\langle \Psi_0 \vert \hat H \vert \Psi_0 \rangle} {\langle \Psi_0 \vert \Psi_0 \rangle} = E_0$$

물론 최소를 찾으려면, 파동함수를 구성하는 파라미터들로 미분을 통해 구할 수 있다. 

이 변분 Thrm. (Rayleigh-Ritz variational theorem)은, 분자 전자구조의 헤밀토니안에서는 만족한다. 

### 5.1.2. Variational Monte Carlo (VMC) methods

VMC 방식은 앞서 설명한 Rayleigh-Ritz variational theorem 과 and Monte Carlo integration methods을 기반으로 만들어졌다. 

변분원리에서 이러한 약속은 분자 전자구조의 헤밀토니안에서는 만족한다.  

이러한 방식에서 에너지의 기댓값을 먼저 쓰고 시작하겠다. 
$$E_\Psi = \langle \Psi \vert E \vert \Psi \rangle = \underline{ \frac{\int {\vert \Psi \vert}^2 [\Psi^{-1} \hat H \Psi] d\tau}{\int {\vert \Psi \vert}^2 d\tau}}\geq E_0$$

저 밑줄친 부분이 기존의 Rayleigh-Ritz variational theorem 과 달라졌다. 저건 약간의 식조작을 통해 얻어진다. 

기존의 Rayleigh-Ritz variational theorem 에서 
$$E_\Psi = \langle \Psi \vert E \vert \Psi \rangle = \frac{\int \Psi^\dagger \hat H \Psi d\tau}{\int \Psi^\dagger  \Psi d\tau} \geq E_0 --- (기존의 Rayleigh-Ritz variational theorem)$$

아래의 두 정의를 사용하여 식을 정리하게된다.

$$\Psi^\dagger  \Psi ={\vert \Psi \vert}^2 $$
$$\Psi^{-1} \Psi = \mathbb{I} $$

그렇게 되면 아래와 같이 우리가 VMC 에서 정의한 형태를 얻을 수 있다. 
$$ \frac{\int \Psi^\dagger \hat H \Psi d\tau}{\int \Psi^\dagger  \Psi d\tau} = \frac{\int \Psi^\dagger \textcolor{red}{\mathbb{I}} \hat H \Psi d\tau}{\int \Psi^\dagger  \Psi d\tau} = \frac{\int \Psi^\dagger \textcolor{red}{\mathbb{ \Psi} \Psi^{-1}} \hat H \Psi d\tau}{\int \Psi^\dagger  \Psi d\tau} = \frac{\int {\vert \Psi \vert}^2 [\Psi^{-1} \hat H \Psi] d\tau}{\int {\vert \Psi \vert}^2 d\tau}$$

이러한 형태에서, 적분부분을 아래와 같이 두가지의 형태로 나누어 생각 할 수 있다. 

$$\frac{\int \textcolor{green}{ {\vert \Psi \vert}^2 } \textcolor{lightblue}{[\Psi^{-1} \hat H \Psi]} d\tau}{ \textcolor{green}{\int {\vert \Psi \vert}^2 d\tau}}$$

초록색으로 표현된 부분은, $\frac{특정 에너지의 확률}{전체 확률} $ 로 보고자 하는 시스템에서 특정 에너지를 관측할 확률이고 ($\therefore \textcolor{green}{\frac{{ {\vert \Psi \vert}^2 }}{\int {\vert \Psi \vert}^2 d\tau}} := \mathcal{P}(\tau)$)

하늘색으로 표현된 부분을 특정 $\tau$ 에서의 에너지이다. ($\therefore \textcolor{lightblue}{[\Psi^{-1} \hat H \Psi]} := E(\tau)$)

즉, 우리가 구하고자 하는 기댓값을, 고전적인 확률의 정의로 쓸 수 있다.
$$ E_\Psi = \int  E(\tau) \mathcal{P}(\tau) d\tau $$

이제 이 식에 약간의 근사(Metropilis-Hastings (MH) algorithm)를 가할것이다. 이 근사는, 수학적으로는 연속적인 적분을 이산적인 M개의 점에 대한 급수로 바꾸게 된다. 여기서 적분 항에 들어있던 확률 분포 $\mathcal{P}(\tau) $ 는 $(\tau)_k $로 변하여, 각 k번째 점의 분포를 나타내는 식으로 바뀌게 된다. 

$$ E_\Psi \approx \frac{1}{M} 	\sum_{k=1}^M  E(\tau)_k  $$


확률 분포 $\mathcal{P}(\tau) $ 는 $(\tau)_k $로 바꿀때  Markov chain Monte Carlo (MCMC) 방식을 사용하여 확률 분포를 M점들의 분포로 바꾸게 된다. 

예시로 아래의 확률분포를 샘플링 해보자. 
     $$ \mathcal{P}(x) = \begin{cases}
            0, & x < 0 \newline
            e^{-x}, & x \geq 0 
            \end{cases} $$

여기서 MH 알고리즘과 "랜덤워크" 커널 (y = x + N(0,1)) (N : 정규분포), 그리고 수용 확률은 $A = min (1, \frac{\mathcal{p}(y)}{\mathcal{p}(x_t)})$ 을 사용한다. 

이 예제 코드는 아래와 같다. 

In [ ]:
def p(x):
  if x < 0:
    y = 0
  else:
    y = np.exp(-x)
  return(y)

n = 10000 # Size of the Markov chain stationary distribution

# Use np.linspace to create an array of n numbers between 0 and n
index = np.linspace(0, n, num=n)
x = np.linspace(0, n, num=n)

x[0] = 3     # Initialize to 3
for i in range(1, n):
  current_x = x[i-1]

  # We add a N(0,1) random number to x
  proposed_x = current_x + stats.norm.rvs(loc=0, scale=1, size=1, random_state=None)

  A = min(1, p(proposed_x)/p(current_x))

  r = np.random.uniform(0,1) # Generate a uniform random number in [0, 1]

  if r < A:
    x[i] = proposed_x       # Accept move with probabilty min(1,A)
  else:
    x[i] = current_x        # Otherwise "reject" move, and stay where we are

plt.plot(index, x, label="Trace plot")
plt.xlabel('Index')
plt.ylabel('MH value')
plt.legend()
plt.show()

위 그림은, 각 k 점이 어떤 x값(MH value) 를 가지는지 나타내는 그림이다. 

즉, 확률 분포를 M개의 k점의 분포로 변환 하였다. 

이러한 확률분포를 통해 주어진 샘플에서, 최적의 bin의 width 를 찾기위한 Freedman–Diaconis rule 을 이용해 최적의 bin 값을 계산한다.
$$ Bin width = \frac {2IQR(x)}{\sqrt{N}^{1/3}}$$


In [ ]:
q25, q75 = np.percentile(x, [25, 75]) #계산에 필요한 사분위수의 범위(IQR)의 두배 를 계산한다. 
bin_width = 2 * (q75 - q25) * len(x) ** (-1/3) #Freedman–Diaconis rule 을 이용해 최적의 bin 값을 계산한다.
bins = round((x.max() - x.min()) / bin_width) #분포의 최대값과 최솟값의 차이를 bin값으로 나눠 필요한 bin의 개수를 구한다. 
print("Freedman–Diaconis number of bins:", bins)

그럼 아래와 같이 Markov Chain의 x값에 대한 확률 분포 그림을 그려 볼 수 있다. 

그리고 이 그림은, 앞서 정의한 확률분포와 어느정도 비슷하다는것을 알 수 있고, 따라서 markov chain은 분포 $\mathcal{p}(x)$ 에 대해 좋은 근사를 보여준다는 것을 알 수 있다. 

In [ ]:
def pp(x):
    return np.exp(-x)
X = np.arange(0,np.max(x),0.05)
plt.hist(x, density=True, bins=bins, label = 'MCMC distribution')
plt.plot(X, pp(X), label='probablity')
plt.ylabel('Density')
plt.xlabel('x')
plt.legend()
plt.show()

In [ ]:
def run_PySCF(info_dict, pyqmc=True, show=True):
  # Reset the files
  for fname in ['mf.hdf5','optimized_wf.hdf5','vmc_data.hdf5','dmc.hdf5']:
    if os.path.isfile(fname):
        os.remove(fname)
  
  atoms = info_dict['atoms']
  coords = info_dict['coords']
  charge = info_dict['charge']
  multiplicity = info_dict['multiplicity']
  atom_pair = info_dict['atom_pair']

  s = ''
  k = 0
  for atom in atoms:
    s += atoms[k] + ' ' + str(coords[k][0]) + ' ' + str(coords[k][1]) + ' ' + str(coords[k][2]) + '; '
    k += 1
  s = s[0:-2]
  
  mol_PySCF = gto.M(atom = s)  
  mf = scf.RHF(mol_PySCF)
  mf.chkfile = "mf.hdf5"
  
  conv, e, mo_e, mo, mo_occ = scf.hf.kernel(mf)
  if show:
    if conv:
      print("PySCF restricted HF (RHF) converged ground-state energy: {:.12f}".format(e))
    else:
      print("PySCF restricted HF (RHF) ground-state computation failed to converge")

  if pyqmc:
    pyq.OPTIMIZE("mf.hdf5",# Construct a Slater-Jastrow wave function from the pyscf output
      "optimized_wf.hdf5", # Store optimized parameters in this file.
      nconfig=100,         # Optimize using this many Monte Carlo samples/configurations
      max_iterations=4,    # 4 optimization steps
      verbose=False)

    with h5py.File("optimized_wf.hdf5") as f:
      iter = f['iteration']
      energy = f['energy']
      error = f['energy_error']
      l = energy.shape[0]
      e = energy[l-1]
      err = error[l-1]
      if show:
        if err < 0.1:
          print("Iteration, Energy, Error")
          for k in iter:
            print("{}:         {:.4f} {:.4f}".format(k, energy[k], error[k]))
          print("PyQMC Monte Carlo converged ground-state energy: {:.12f}, error: {:.4f}".format(e, err))
        else:
          print("PyQMC Monte Carlo failed to converge")

  return conv, e

### 5.1.3. Quantum Phase Estimation (QPE)

양자 화학적인 계산에서, 화학반응에서의 각 분자간의 전체 전기적 에너지의 계산은 매우 높은정밀도를 필요로한다. 

여기서 "Quantum Phase Estimation (QPE)" 알고리즘은 주어진 오차 내에서 양자적인 시뮬레이션을 가능하게 하는 매우 유니크한 특성을 가진다. 

이 알고리즘이 미래에 만들어질 Fault-tolerant 양자컴퓨터가 만들어졌을때 매우 효율적으로 사용할 중요한 알고리즘 중 하나이다. (아직 NISQ에서는 불가능하다.) 

고유상태가 $\vert \psi \rangle $ 로 주어지고, 그의 고유치가 $e^{2 \pi i \theta}$ 로 주어지는 어떤 유니터리 연산자가 있다고 하자. 그럼 아래의 고유치 문제를 만들 수 있을것이다. 
$$ U\vert \psi \rangle =e^{2 \pi i \theta} \vert \psi \rangle $$

상태 $\vert \psi \rangle $ 를 준비하고, 그 상태에 U 를 가하는데에 QPE 알고리즘은 $2^n \theta$ 를 계산하게된다. (여기서 n은 $ \theta $ 를 추정하는데에 사용하느 큐비트의 개수이다. 그리고 $\theta$ 에 해당하는 만큼 원하는 정확도를 얻을 수 있다.)




모든 게이트는 사실 양자역학적인 Time Evolution이다. 

그리고 그 Time Evolution은 아래와같이 Shrodinger 방정식으로 기술 된다. 
$$i\hbar \frac{d}{dt} \vert \psi \rangle = \hat H \vert \psi \rangle$$

이 방정식에서 $\hat H$ 이 시간에 무관한 경우의 솔루션은 아래와 같다. 
$$ \vert \psi \rangle = e^{\frac{t}{ih} \hat H}\vert \psi_0 \rangle$$
$$where,  \vert \psi_0 \rangle  = Initial Condition (or some steady state)$$
여기서 아래와 같이 "time evolution" 연산자 $U(t)$ 를 정의하게 되면
$$ U(t) =: e^{\frac{t}{ih} \hat H}$$
임의의 시간(t)에 대한 상태 $\vert \psi(t) \rangle$ 는 아래와 같은 Time Evolution으로 나타낼 수 있다. 
$$\vert \psi(t) \rangle = \hat U(t) \vert \psi_0 \rangle $$

이제 이러한 이론적인 정의를 바탕으로 Qikit에서 제공하는 QPE 클래스를 이해해보자. 

1) $\hat U(\theta)$ 를 정의한다. 
    이때 $\hat U(\theta)$ 아래의 식을 만족하다. 
    $$U(\theta)\vert q_0 \rangle = e^{2 \pi i \theta} \vert q_0 \rangle = p(2\pi \theta)\vert q_0 \rangle $$
    여기서 $p(\lambda)$ 는 Phase gate로 아래의 행렬형태를 가진다. 
    $$p(\lambda)= \begin{pmatrix} 1 & 0 \\ 0 & e^{i\lambda} \end{pmatrix}$$

    이 P gate를 이용해 $U(\theta)$ 를 코드로 정의해보자. 

In [ ]:
def U(theta):
  unitary = QuantumCircuit(1)
  unitary.p(np.pi*2*theta, 0)
  return unitary

이제 본격적으로 QPE 를 해볼것인데, 
Qiskit에서 제공하는 QPE 클래스에서는 아래의 3개의 파라미터가 필요하다. 

1) Unitary : Time evolution 과 관련된 연산자를 어떤 연산자를 사용할 것인가. 
2) nqubit : 큐비트의 수 ; 여기서는 3으로 정의한다. 
3) show (boolean) : QPE 이후의 위상 변화를 보여준다.

이를 이용해 do_qpe 함수를 만들자. 

In [ ]:
def do_qpe(unitary, nqubits=3, show=True):
  state_in = QuantumCircuit(1)
  state_in.x(0)
  pe = PhaseEstimation(num_evaluation_qubits=nqubits, sampler = Sampler())
  result = pe.estimate(unitary, state_in)
  phase_out = result.phase
  if show:
    print("Number of qubits: {}, QPE phase estimate: {}".format(nqubits, phase_out))
  return(phase_out)

In [ ]:
#이 함수를 이용해 테스트를 해보자. 
theta = 0
for k in range(10):
    r = (1/2)**(k+1)
    theta += r


print("theta: {}".format(theta))
unitary = U(theta)
result = do_qpe(unitary, nqubits=6)

위의 코드에서 Theta 의 크기 부분이나, 큐비트의 개수를 조절하며 확인해보면 
큐비트의 개수와 계산값의 오차가 연관되어있는것을 알 수 있다. 

### 5.1.4. Description of the VQE algorithm

VQE란 무엇인가를 생각해보자. 
사실, 이상적인 양자하드웨어 (이하, QPU)가 개발이 된다면 VQE를 할 필요도 없이 QPE를 통해 우리가 원하는 해를 얻을 수 있다. 하지만, 현재의 QPU 는 에러를 갖고있고, 큐비트의 수가 제한적인 NISQ 장비를 가지고 있다. 이러한 상황에서 이상적인 하드웨어를 필요로하는 알고리즘이 굳이 필요하나? 지금 당장 사용 할 수 있는 알고리즘은 없을까? 하는 생각에서  고전적인 컴퓨팅에서 잘하는 "최적화" 그리고 QPU에서 잘하는 "중첩상태의 측정" 이 두가지 장점을 살려 개발된것이 바로 VQE 이다.

루프 안에서 고전적인 알고리즘에서는 분자의 바닥상태(가장 낮은 에너지를 갖는 상태)를 찾기 위해 주어진 목적함수에 대해 양자회로의 파라미터를 최적화 한다.
파라미터화 된 양자 회로(PQC ; Parameterized Quantum Circuit)는 시험 양자상태를 Trial Solution(Ansatz) 으로 준비한다. 그 양자회로를 반복해서 측정함으로써, 우리는 준비된 Trial State 에 대한 에너지 observable의 기댓값을 측정 할 수 있다. 

VQE 알고리즘은 헤밀토니안 $\hat H$ 로 인코딩된 어떤 시스템에 대해 가장 낮은 에너지 $E_0$를 가지는 바닥상태에 가까운 값을 제공한다. 예를들면 분자의 바닥상태 에너지같은것이 있다. 여기서는 PQC($\vert \Psi (\theta)\rangle $) 의 $\theta$  를 $E_{\Psi(\theta)}$ 가 최소가 되도록 최적화 하게된다. 즉, 아래의 식이다. 
$$E_0 \geq E_{\Psi(\theta)} = \langle \Psi(\theta)\vert \hat H \vert \Psi(\theta) \rangle $$

여기서의 헤밀토니안은, 앞선내용을 이용하자면, Fermionic 생성\소멸 연산자로 매핑할 수 있고, 그 연산자들을 JW Transform 등의 방법을 통해 큐비트 버전의 생성/소멸 연산자, 혹은 파울리 게이트를 통해 표현 할 수 있다. 즉, 헤밀토니안을 파울리 연산자의 곱으로 이루어진 항들의 합으로 표현 할 수 있다. 
$$\hat H = \sum_{k=l}^{M-1}c_kP_k$$
- 여기서 M 은 더해지는 총 항의 개수이다. (Fermionic Operator ver. 의 헤밀토니안을 생각한다면 쉽게 생각 할 수 있다.)
- $c_k$ 는 각 항들의 가중치 (마찬가지로 Fermionic Operator ver. 의 헤밀토니안에서의 $h_{pq}$ , $h_{pqrs}$에 해당한다.) 
- $P_k$ 는 각 항의 파울리 곱을 의미한다. ($a^{\dagger}_0a_o -> \underline{II}$)
$P_k$를 좀더 풀어서 적게되면 아래와 같이 적을 수 있다. 
$$\hat H = \sum_{k=l}^{M-1}c_k [\bigotimes^N_j \sigma_{k,j}]$$
- 여기서 N은 큐비트의 개수
- 뒤의 $P_k$에 해당하는 부분에 해설을 달자면, $\sigma_{k,j}$는 k번째 항에서 j번째 큐비트에 작용하는 연산자를 나타낸다. ($\sigma_{k,j} \in 	\{I, \sigma_x , \sigma_y, \sigma_z \}$)

따라서 에너지의 기댓값은 아래와 같이 각각의 파울리스트링(파울리 스트링의 텐서곱)의 측정을 통해 측정 될 수 있다.
$$E_{\Psi(\theta)}= \langle \Psi(\theta)\vert \hat H \vert \Psi(\theta) \rangle $$ 
$$ \begin{matrix}
E_{\Psi(\theta)} &=& \langle \Psi(\theta)\vert \hat H \vert \Psi(\theta) \rangle \\
       &=& \sum_{k=l}^{M-1} c_k \langle \Psi(\theta)\vert P_k \vert \Psi(\theta) \rangle \\
       &=& \sum_{k=l}^{M-1} c_k \langle \Psi(\theta)\vert \bigotimes^N_j \sigma_{k,j} \vert \Psi(\theta) \rangle
\end{matrix} $$

그럼 시험상태 $\vert \Psi(\theta) \rangle$ 는 어떻게 만드는가? 
일단 위 식의 저 $\theta$ 는 이제 양자회로의 파라미터이다. 정확히는 어떠한 Basis를 이용해 양자 회로의 뼈대를 구성하고, 그 basis의 계수들을 파라미터로 하면, 그 basis에서 표현할 수 있는 Hilbert 공간을 모두 표현 할 수 있다. 즉, 계수를 조절(최적화)함으로 모든 상태를 구현 할 수 있게된다. 그렇게 상태를 조절 할 수 있도록하는것이 $\theta$ 로 표현되는 파라미터이다. 
그 $\theta$ 는 당연히 하나가 아닌 여러개일것이다. 파라미터가 여러개일수록 더 많은 basis가 있는것이므로, 더 정확한 답을 얻을 수 있겠지만, 파라미터가 늘어날수록, 최적화에 필요한 Cost가 늘어나게된다. 따라서 적당한 파라미터의 개수가 필요하다. 
$$\theta = (\theta_0 , \theta_1, ... , \theta_m)$$

그러한 $\theta$로 구성된 양자상태를 만들기 위해서는 모든 N개의 큐비트가 $\vert 0 \rangle $로 초기화된 상태에 우리가 원한는 양자상태로 바꾸어주는 유니터리 연산자 $U(\theta)$ 를 가하여 시험 상태를 만들게 된다.
$$\vert \Psi(\theta) \rangle = U(\theta)\vert 0 \rangle^{\otimes N}$$
기댓값의 계산을 위해 $\langle \Psi(\theta) \vert$ 을 구하려면 Dirac Notation의 정의에 의해 $\vert \Psi(\theta) \rangle ^\dagger$ 를 구하게되면 아래와 같다.
$$ \begin{matrix}
\langle \Psi(\theta) \vert &=& \vert \Psi(\theta) \rangle ^\dagger \\
       &=& \langle 0 \vert ^{\otimes N} U(\theta)^{\dagger}
\end{matrix} $$

이렇게 되면 기댓값을 적어 볼 수 있게된다. 

$$ \begin{matrix}
E_{\Psi(\theta)} &=& \langle \Psi(\theta)\vert \hat H \vert \Psi(\theta) \rangle \\
       &=& \langle 0 \vert ^{\otimes N} U(\theta)^{\dagger} \hat{H} U(\theta) \vert 0 \rangle^{\otimes N} \\
       &=& \sum_{k=l}^{M-1} c_k \langle 0 \vert ^{\otimes N} U(\theta)^{\dagger} P_k U(\theta) \vert 0 \rangle^{\otimes N} \\
       &=& \sum_{k=l}^{M-1} c_k \langle 0 \vert ^{\otimes N} U(\theta)^{\dagger} \bigotimes^N_j \sigma_{k,j} U(\theta) \vert 0 \rangle^{\otimes N} 
\end{matrix} $$

#### 이제 각 $P_k$ 에 대해 기댓값을 측정하고, 그걸 합하여, 전체 헤밀토니안의 기댓값을 측정해보자. 
각 $P_k$ 에 따라 $U(\theta)$ 를 아래와같이 "회전"하여 양자 회로를 돌리게된다 (즉,상태, 헤밀토니안을 만들고 기댓값을 측정하게된다.)
"회전"은 무엇인지 짚고 넘어가자면, 주어진 $P_k$ 에 따라 효율적인 측정 방향에 따라 측정 전에 회로에 $R_k \in \{\mathbb{I}, R_X(-\pi/2), R_Y(\pi/2)\}$ 를 가하여, 측정에 유리한 방향으로 회전시키게된다.

각 basis가 의미한는것은 아래의 bloch 구 에서 자세히 알아볼 수 있다. 

![bloch_sphere](./img/bloch_sphere.jpeg)

하나의 큐비트는 위 그림에서 표현한 bloch 상의 한 단위벡터로 표현 될 수 있다. 
- 길이는 1로 고정
- $\theta$ $\phi$ 로 방향을 설정
이러한 근거로 우리는 하나의 임의 상태를 아래와 같이 표현 할 수 있는것이다. 
$$\vert \Psi \rangle = \cos(\pi/2) \vert 0 \rangle+ i\sin(\pi/2) e^{i\phi} \vert 1 \rangle$$

다시 원래의 얘기로 돌아와서 basis를 회전시키는 이유는, 우리는 지금까지 $\vert 0 \rangle$,  $\vert 1 \rangle$ 상태를 이용해서 양자상태를 표현했다. 이는 일반적으로 사용되는 방법으로 Z-basis로 표현한 경우이다. 하지만 이 외에도, 설계상의 이유로 $\vert + \rangle$ 와 $\vert - \rangle$ 로 표현되는 X-basis나, $\vert +i \rangle$ 와 $\vert -i \rangle$ 상태로 표현되는 Y-basis로 측정을 하는것이 유리 할 수도있다. 하지만 양자컴퓨터에서 특정 방향으로 측정을 할 수 없기에, 양자상태를 Rotation Gate($R_x, R_y $) 를 이용해서 회전시키고 측정하게 된다. 




고전적인 컴퓨터에선 이제 계산된 기댓값들을 주어진 가중치에 대한 가중치합을 하게된다. 

그리고 가중치합을 통해 계산한 $E_{\Psi(\theta)}$ 에 대해 $\theta$ 를 고전적인 방법으로 최적화하게 되고, $E_{\Psi(\theta)}$가 수렴할때 까지 최적화 하게된다. 

그렇게 최적화를 통해 얻은 파라미터 Set $\theta_{min}$ 를 통해 시험함수를 표현하게되면 그 상태가 근사적으로 바닥상태를 나타내게된다. 
이 알고리즘의 요약은 아래의 그림과 같다. 

![VQE_Pipeline1](./img/VQE_Pipeline1.png)

![VQE_Pipeline2](./img/VQE_Pipeline2.png)

#### Trial wavefunctions

Coupled-Cluster(CC) 이론은 multi-electron wave function을 ($\Psi$) exponential cluster operator $\hat{T} = \hat{T}_1 + \hat{T}_2 + ... + \hat{T}_n$을 상용하여 구성하는 이론으로, 여기에서 $\hat{T}_1$은 모든 single excitation의 operator이고 $\hat{T}_2$은 모든 double excitation의 operator를 말한다. 하지만 exponential cluster operator는 unitary operator가 아니여서 state에 가해줄 수 없으므로 이것을 unitary operator로 바꿔주는 과정을 거쳐 사용한다. 이것을 unitary Coupel-Cluster(UCC) ansatz라 한다. UCC ansatz를 이용하여 우리가 원하는 양자 상태 $|\Psi(\theta) \rangle$를 얻을 수 있다.
$$
|\Psi(\theta) \rangle = exp[\hat{T}(\theta)-\hat{T}^\dagger(\theta)]|\Psi_{ref}\rangle
$$
위의 식에서 $|\Psi_{ref}\rangle$은 Hartree-Fock ground state이다.
그리고 UCC method에서 single과 double excitation만으로 제한하면(UCCSD) $\hat{T}_1$과 $\hat{T}_2$는 다음과 같이 쓸 수 있다.
$$
\hat{T}_1(\theta) = \sum_{i;m}\theta^m_{i} a^\dagger_m a_i
$$
$$
\hat{T}_2(\theta) = \sum_{i,j;m,n}\theta^{m,m}_{i,j} a^\dagger_n a^\dagger_m a_j a_i
$$

위 식에서 $a^\dagger_m$과 $a_i$는 각각 fermionicn creation operator, fermionic annihilation operator이고 $\theta$는 모든 expansion coeffieient를 모은 parameter set이다.

UCCSD ansatz는 전 chater에서 본 mapping들로 qubit operator로 맵핑할 수 있다.

#### Setting up the VQE solver

다음으론 VQE를 구성할 때 사용할 VQE solver의 setting이다.

먼저 다음과 같이 NumPyminmum eignesolver를 setting한다. NumPyminmum eignesolver의 자세한 사항을 알고 싶으면 ReF.[Julien Gacon, Lab 4: Introduction to Training Quantum Circuits, Qiskit Summer School 2021, https://learn.qiskit.org/summer-school/2021/lab4-introduction-training-quantum-circuits]을 확인해 보라.

In [ ]:
numpy_solver = NumPyMinimumEigensolver()

다음 코드는 Two-Local circuit의 setting이다.

In [ ]:
tl_circuit = TwoLocal(rotation_blocks = ['h', 'rx'], entanglement_blocks = 'cz',
                      entanglement='full', reps=2, parameter_prefix = 'y')

다음 코드는 위의 Two_local circuit으로 solver를 구성하는 과정이다. optimazer는 SPSA를 사용한다.

In [ ]:
estimator = Estimator()
optimizer = SPSA(maxiter=100)
vqe_tl_solver = VQE(estimator, tl_circuit, optimizer)

Quantum Natural SPSA(QN-SPSA) optimazer에서 결과를 추가하기 위해 loss array인 qnspsa_loss를 만들고 얻은 값을 추가해주는 함수이다.

In [ ]:
qnspsa_loss = []
def qnspsa_callback(nfev, x, fx, stepsize, accepted):
    qnspsa_loss.append(fx)

다음 코드는 bopes() 함수를 정의하는 코드이다.

BOPES(Born-Oppenheimer pothential energy surface)를 얻으려면 energy surface를 얻으려면 원자의 거리를 바꾸어 가며 그때의 에너지를 얻어야 한다. bopes() 함수는 이것을 돕기 위한 함수이다.

bopes() 함수는 먼저 결합될 원자쌍의 좌표를 저장한다. 그 좌표의 x값의 차이가 무시할 수 없을 만큼 크면 원자싸의 좌표의 x,y 평면의 선분을 얻는다. 이 과정은 원자의 거리를 바꿀때 x의 값에 따라 y의 값을 바꿔주기 위함이다. 다음으로 만약 이동시킬 원자의 x, y좌표가 무시할 수 있을 정도로 작다면 원자의 z축만 조금씩 이동시키고 z좌표가 작다면 원자의 x축을 이동시키고 x축의 이동에 따른 y값을 바꿔준다. 이런 새로운 좌표값으로 ground energy를 풀고 각 조건에 따로 좌표를 조금씩 바꾸는 것을 반복하여 전체 energy surface를 얻는다.

In [ ]:
# Sampling the potential energy surface
_EPS = 1e-2 # Global variable used to chop small numbers to zero
def bopes(info_dict, mapper_name, num_electrons, num_spatial_orbitals, two_qubit_reduction, z2symmetry_reduction,
          name_solver, perturbation_steps, mapper, solver, show=True):
  
  atoms = info_dict['atoms']
  coords = info_dict['coords']
  charge = info_dict['charge']
  multiplicity = info_dict['multiplicity']
  atom_pair = info_dict['atom_pair']

  size = len(perturbation_steps)
  
  energy = np.empty(size)

  x0 = coords[atom_pair[0]][0]
  y0 = coords[atom_pair[0]][1]
  z0 = coords[atom_pair[0]][2]
  if show:
    print("x0, y0, z0 :", x0, y0, z0)
  
  x1 = coords[atom_pair[1]][0]
  y1 = coords[atom_pair[1]][1]
  z1 = coords[atom_pair[1]][2]
  if show:
    print("x1, y1, z1 :", x1, y1, z1)
    
  if abs(x1 - x0) > _EPS:
    # Find the equation of a straight line y = m*x + p that crosses the points of the atom pair
    m = (y1 - y0)/(x1 - x0)
    p = y0 - m*x0  
  
  for k in range(size):
    if (abs(x0)<_EPS and abs(y0)<_EPS):
      z0_new = z0 + perturbation_steps[k]
      coords_new = []
      for l in range(len(coords)):
        if l == atom_pair[0]:
          coords_new.append((0.0, 0.0, z0_new))
        else:
          coords_new.append(coords[l])

    elif (abs(z0)<_EPS and abs(z1)<_EPS):
      x0_new = x0 + perturbation_steps[k]
      y0_new = m*x0_new + p
      coords_new = []
      for l in range(len(coords)):
        if l == atom_pair[0]:
          coords_new.append((x0_new, y0_new, 0.0))
        else:
          coords_new.append(coords[l])
    
    info_dict_new={'atoms':atoms, 'coords':coords_new, 'charge':charge, 
                  'multiplicity':multiplicity, 'atom_pair':atom_pair}
    
    fermionic_hamiltonian, num_particles, num_spin_orbitals, qubit_op, mapper, ground_state = \
    solve_ground_state(info_dict_new, mapper_name=mapper_name, num_electrons=num_electrons, num_spatial_orbitals=num_spatial_orbitals,
                  two_qubit_reduction=two_qubit_reduction, z2symmetry_reduction=z2symmetry_reduction, 
                  name_solver=name_solver, solver=solver, pyqmc=False)
    
    energy[k] = ground_state.total_energies
    
  return perturbation_steps, energy

## 5.2. Example chemical calculations

앞선 Chapter 4, Molecular Hamiltonians에선 핵의 운동의 potential energy surface(PES)의 근사는 BO 근사를 사용하였기 때문에 발생했다. 우리는 실험적 데이터와 컴퓨터 시뮬레이션을 통해 준경험적 PES 근사 방법을 사용할 것이다.


PES는 산과 협곡의 지형과 비교할 수 있다. 실제 화학에서 우리는 PES의 전체 최소(ocean floor)을 찾기 원하지 지역 최소(mountain meadows)을 찾기 원하는 것이 아니다. 우리는  전체 최소를 찾기 위해 variational method로 기존컴퓨터와 양자 컴퓨터를 사용한다. 이것은 지형 주위에 공을 굴리는 것과 같다. 만약 공이 어떤 방향으로 살짝 움직이면 일반적으로 최소에 다다를 때까지 내려가게 된다. 우리는 이것을 gradient descent라 부른다. gradient descent는 수치적인 변화 입력 값이나 PES를 묘사하는 wavefunction의 식을 분석하여 얻는다.


PES를 결정하는 계산에 단계에서 우리는 에너지의 전체 최소를 찾기 위해 계산이 최적화된 시험 wave function을 예측한다. 우리는 이것을 주어진 eigen valuedptj 가능한 가장 낮은 에너지의 전체 최소라 부른다.
우리는 classical에  PySCF RHF, PyQMC variational Monte Carlo,  QPE, PySCF driver로 STO-3G basis를 사용한 Qiskit Nature의 VQE로 ground state를 푸는 것과 3개의 분자의 BOPES를 그리는 것의 몇개의 구현을 볼 것이다. 

이번 section에서 다음을 주제를 다룬다.
- Section 5.2.1 수소분자
- Section 5.2.2 LiH 분자
- Section 5.2.3 대규모 분자 

우리는 $\mathsf {get\_particle\_number()}$ 함수를 사용하여 주어진 전자 구조 문제의 입자 수의 특성을 얻는 수소분자의 fermionic Hamiltonian operator를 구성한다. 다음 코드를 확인하면 get_particle number() 함수는 주어진 problem의 num_spin_orbitals와 num_particles를 저장하고 출력한다. show=True 이면 print한다.

In [ ]:
def get_particle_number(problem, show=True):
  
  # https://qiskit.org/documentation/nature/stubs/qiskit_nature.second_q.problems.ElectronicStructureProblem.num_spin_orbitals.html
  num_spin_orbitals = problem.num_spin_orbitals
  num_particles = problem.num_particles
  
  if show:
    print("Number of particles: {}".format(num_particles))
    print("Number of spin orbitals: {}".format(num_spin_orbitals))
    
  return num_particles, num_spin_orbitals

그리고  fermionic operator를 qubit operator로 변환하기 위해 Qiskit Nature에 qubit Hamiltonian을 구성하는  $\mathsf {fermion\_to\_qubit()}$ 함수를 사용한다.
- $\mathsf {second\_q\_op}$ : fermionic operator이다.
- mapper_name : jordan-Wigner, Parity, Bravyi-Kitaev 등을 사용할 수 있다.
- truncate : Pauli list를 줄이기 위한 정수. 디폴트는 20 항목이다.
- two_qubit_reduction : Boolean 자료를 받는다. 디폴트는 False이고, two-qubit reduction의 실행을 결정한다.
- z2symmetry_reduction : 디폴트는 None으로, Z2 symmetry reduction이 resultin qubit operators에 적용할 것인지를 나타낸다. 이것은 operator에서 찾을 수 있는 수학적인 symmetry를 기반으로 계산된다.
- show : True로 setting하면 transformation과 결과들의 이름을 보여준다.

fermion_to_qubit() 함수는 먼저 mapper_name으로 원하는 mapping mapper로 지정하고 이것을 주어진 fermionic operator 즉, secound_q_op에 원하는 mapping을 적용하여 fermionic operter를 qubit operator로 변환하고 이 값을 qubit_op에 저장한다. 이후 show와 z2symmetry_reduction을 주어진 boolean 값으로 적용 여부를 선택하여 적용한다. 이후 truncate의 값에 따라 qubit_op의 list를 짤라서 print한다. 정확한 이유는 모드지만 fermion_to_qubit() 함수에선 two_qubit_reduction 변수를 사용하지 않는다.

In [ ]:
def fermion_to_qubit(problem, second_q_op, mapper_name, truncate=20, two_qubit_reduction=False, z2symmetry_reduction=None, show=True): 
# Electronic Structure Problems with v0.5
# https://qiskit.org/documentation/nature/migration/00b_Electronic_structure_with_v0.5.html
# https://qiskit.org/documentation/nature/stubs/qiskit_nature.second_q.mappers.QubitConverter.html#qubitconverter
# https://qiskit.org/ecosystem/nature/migration/0.6_c_qubit_converter.html
  if show:
    print("Qubit Hamiltonian operator")
    print("{} transformation ". format(mapper_name))

  match mapper_name:
    case "Jordan-Wigner":
      mapper = JordanWignerMapper()
    case "Parity":
      mapper = ParityMapper(num_particles=problem.num_particles)
    case "Bravyi-Kitaev":
      mapper = BravyiKitaevMapper()
  
  qubit_op = mapper.map(second_q_op)
  
  if z2symmetry_reduction != None:
    tapered_mapper = problem.get_tapered_mapper(mapper)
    qubit_op = tapered_mapper.map(second_q_op)
    
  n_items = len(qubit_op)
  if show:
    print("Number of items in the Pauli list:", n_items)
    if n_items <= truncate:
      print(qubit_op)
    else:
      print(qubit_op[0:truncate])
  return qubit_op, mapper

Qiskit Nature는 GroundStateEigensolver class를 지원한다. 이 class는 분자의 ground state를 계산한다. 
그리고 다음과 같은 input parameter를 사용하는 run_vqe() 함수를 정의할 것이다. 
- name : 'Numpy exact solver'와 같은 character들의 문자를 프린트한다.
- problem : 위에서 말한 풀 문제의 분자의 특성
- qubit_converter : JordanWignerMapper(), ParityMapper(), BravyiKitaevMapper()로 fermion_to_qubit() 함수의 결과이다.
- solver : section 5.2.3에서 정의한 solver중 하나. numpy_solver, vqe_tl_solvr. 여기에서 UCCSD solver는 따로 정의하지 않고 내장된 solver를 사용하여 따로 지정해 주지 않아도 된다.

run_vqe() 함수는 먼저 주어진 qubit_converter와 solver를 입력하여 Qiskit Nature의 GroundStateEigensolver의 class를 생성한다. 이 class에서 solve 함수로 problem을 풀어 주어진 problem의 ground_state를 풀고 저장한다. 이후 ground_state를 출력한다.

In [ ]:
# Leveraging Qiskit Runtime
# https://qiskit.org/documentation/nature/tutorials/07_leveraging_qiskit_runtime.html
def run_vqe(name, problem, qubit_converter, solver, show=True):
  calc = GroundStateEigensolver(qubit_converter, solver)
  start = time.time()
  ground_state = calc.solve(problem)
  elapsed = str(datetime.timedelta(seconds = time.time()-start))
  if show:
    print("Running the VQE using the {}".format(name))
    print("Elapsed time: {} \n".format(elapsed))
    print(ground_state)
  return ground_state

다음으로 quantum phase estimation 하고 전자의 ground state energy의 추정치로 Hamiltonian의 egenvalue를 얻어내기 위해 run_qpe() 함수를 정의한다. 이 함수는 다음과 같은 input parameter를 가진다. 
- qubit_op : fermion_to_qubit()함수로 얻은 qubit Hmailtonian operator.
- n_ancillae : ancillae qubit의 개수로 정수형을 받고 디폴트는 3이다.
- num_time_slices : 근사 정확도를 향상시키기 위한 Trotterization repetition의 수. Quskit에 PauliTrotterEvolution class를 사용한다. 정수형을 받고 디폴트는 1이다.
- show : 값이 True이면 중간 결과를 보여준다.

run_qpe() 함수는 HamiltonianPhaseEstimation class를 사용하여 주어진 n_ancillae와 qiskit.primitives의 Sampler()를 sampler로 지정하여 qpe class를 생성한다. state_preparation을 만들고 classdml estimate 함수를 이용하여 주어진 qubit_op로 result를 얻는다.

In [ ]:
def run_qpe(qubit_op, n_ancillae=3, num_time_slices=1, show=True):
  
  qpe = HamiltonianPhaseEstimation(n_ancillae, sampler=Sampler())
  state_preparation = None
  result = qpe.estimate(qubit_op, state_preparation, evolution=None)

  if show:
    eigv = np.real(result.most_likely_eigenvalue)
    print("QPE computed electronic ground state energy (Hartree): {}".format(eigv))
  
  return eigv

다음으로 atomic separation 함수로 에너지를 그래프로 그리기 위해 plot_energy_landscape() 함수를 정의한다. 

In [ ]:
def plot_energy_landscape(dist, energy):
  if len(dist) > 1:
      plt.plot(dist, energy, label="VQE Energy")
      plt.xlabel('Atomic distance Deviation(Angstrom)')
      plt.ylabel('Energy (hartree)')
      plt.legend()
      plt.show()
  else:
      print("Total Energy is: ", energy_surface_result.energies[0], "hartree")
      print("(No need to plot, only one configuration calculated.)")
  return

다음으로 plot_loss() 함수를 정의한다. 이 함수는 다음과 같은 input parameter를 가진다.
- loss : 일반적으로 callback 함수로 얻은 loss 값으로 plot을 그릴 대상
- label : loss로 그린 plot의 label.
- target : target의 값의 수평선을 그림.

In [ ]:
def plot_loss(loss, label, target):
  plt.figure(figsize=(12, 6))
  plt.plot(loss, 'tab:green', ls='--', label=label)
  plt.axhline(target, c='tab:red', ls='--', label='target')
  plt.ylabel('loss')
  plt.xlabel('iterations')
  plt.legend()

다음으로 ground state를 푸는 solve_ground_state() 함수를 정의한다. 이 함수는 분자의 구조를 정의하는 다음과 같은 주요 input parameter를 갖는다.
- info_dict : 분자의 구조 dictionary.
- mapper_name :  사용할 mapper의 이름. 디폴트는 Parity이다.
- solver : eigensolver를 지정한다. 디폴트는 NumPyMinimumEigensolver()를 사용한다.

solve_ground_state() 함수는 먼저 info_dict에 담겨있는 분자 구조의 다양한 정보(atoms, coords, charge, multiplicity, atom_pair)를 풀어 각각 변수로 지정한다. 이 변수들을 입력하여 각 정보를 가진 MoleculeInfo class를 생성한다. 다음으로 주어진 분자 정보를 이용하여 분자의 전자 구조 diver를 정의한다. 이때 PySCFDriver class를 사용하고 moleculeinfo class와 basis를 정해서 diver를 생성한다. 이후 PySCFDriver의 run() 함수로 전자 구조 problem을 정의한다. 이 problem과 미리 정한 mapper, name_solver로 위에서 정의했던 run_vqe() 함수를 돌려 전자 구조 problem의 ground state를 얻는다.


In [ ]:
settings.use_pauli_sum_op = False

def solve_ground_state(
    info_dict,
    mapper_name="Parity",
    num_electrons=None,
    num_spatial_orbitals=None,
    freeze_core=None, 
    two_qubit_reduction=False,
    z2symmetry_reduction = "Auto",
    name_solver='NumPy exact solver',
    solver=NumPyMinimumEigensolver(),
    plot_bopes=False,
    perturbation_steps=np.linspace(-1, 1, 3),
    pyqmc=True,
    n_ancillae=3, 
    num_time_slices=1,
    loss=[],
    label=None,
    target=None,
    show=True
):
    
    atoms = info_dict['atoms']
    coords = info_dict['coords']
    charge = info_dict['charge']
    multiplicity = info_dict['multiplicity']
    atom_pair = info_dict['atom_pair']
    
    moleculeinfo = MoleculeInfo(atoms, coords, charge=charge, multiplicity=multiplicity)
    
    # Defining the electronic structure molecule driver
    # Electronic Structure Problems with v0.5
    # https://qiskit.org/documentation/nature/migration/00b_Electronic_structure_with_v0.5.html
    # https://qiskit.org/documentation/nature/tutorials/01_electronic_structure.html
    
    #driver = ElectronicStructureMoleculeDriver(molecule, basis='sto3g', driver_type=ElectronicStructureDriverType.PYSCF)
    driver = PySCFDriver.from_molecule(moleculeinfo, basis="sto3g")

    # Splitting into classical and quantum
    if num_electrons != None and num_spatial_orbitals != None:
      # https://qiskit.org/documentation/nature/tutorials/05_problem_transformers.html
      # https://qiskit.org/documentation/nature/stubs/qiskit_nature.second_q.transformers.ActiveSpaceTransformer.html#activespacetransformer
      split = ActiveSpaceTransformer(num_electrons=num_electrons, num_spatial_orbitals=num_spatial_orbitals)
    else:
      split = None

    # Define an electronic structure problem
    problem = driver.run()
    if split != None:
      problem = split.transform(problem)
    elif freeze_core != None:
      problem = freeze_core.transform(problem)
    
    # Get the electronic energy fermionic Hamiltonian
    fermionic_hamiltonian = problem.hamiltonian
    second_q_op = fermionic_hamiltonian.second_q_op()
    
    if show:
      print("Fermionic Hamiltonian operator")
      # We print the first 20 terms of the fermionic Hamiltonian operator of the molecule
      # https://qiskit.org/documentation/nature/migration/00b_Electronic_structure_with_v0.5.html
      print("\n".join(str(second_q_op).splitlines()[:20] + ["..."]))
    
     # Get number of particles and number of spin orbitals
    num_particles, num_spin_orbitals = get_particle_number(problem, show=show)
    
    # Use the function fermion_to_qubit() to convert a fermionic operator to a qubit operator
    if show:
      print(" ")
    qubit_op, mapper = fermion_to_qubit(problem, second_q_op, mapper_name, two_qubit_reduction=two_qubit_reduction, z2symmetry_reduction=z2symmetry_reduction, show=show)
    
    # Run the the PySCF RHF method
    if show:
      print(" ")
    conv, e = run_PySCF(info_dict, pyqmc=pyqmc, show=show)
    
    # Run QPE
    eigv = run_qpe(qubit_op, n_ancillae=n_ancillae, num_time_slices=num_time_slices, show=show)

    # Run VQE
    if show:
      print(" ")
    
    #https://qiskit.org/ecosystem/nature/howtos/vqe_ucc.html
    if name_solver == 'UCCSD ansatz':
      ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(problem.num_spatial_orbitals,problem.num_particles,mapper),
        )
      solver = VQE(Estimator(), ansatz, SLSQP())
    
    ground_state = run_vqe(name_solver, problem, mapper, solver, show=show)
    # Plot loss function
    if loss != []:
      plot_loss(loss, label, target)
    
    if plot_bopes:
      # Compute the potential energy surface
      dist, energy = bopes(info_dict, mapper_name, num_electrons, num_spatial_orbitals, two_qubit_reduction, z2symmetry_reduction,
          name_solver, perturbation_steps, mapper, solver, show=True)
      
      # Plot the energy as a function of atomic separation
      plot_energy_landscape(dist, energy)

    return fermionic_hamiltonian, num_particles, num_spin_orbitals, qubit_op, mapper, ground_state

### 5.2.1. Hydrogen molecule

위의 코드를 사용하여 수소 분자에 적용한다. 다음과 같이 먼저 수소분자의 기본 정보를 입력해주고 info_dict에 입력하기 위해 dictionary 구조로 만든다.

In [ ]:
# Electronic Structure Problems with v0.5
# https://qiskit.org/documentation/nature/migration/00b_Electronic_structure_with_v0.5.html
H2_atoms = ["H", "H"]
H2_coords = [(0.0, 0.0, 0.0), (0.0, 0.0, 0.735)]
H2_charge = 0
H2_multiplicity = 1
H2_atom_pair=(1, 0)
H2_info_dict={'atoms':H2_atoms, 'coords':H2_coords, 'charge':H2_charge, 'multiplicity':H2_multiplicity, 'atom_pair':H2_atom_pair}

H2_moleculeinfo = MoleculeInfo(H2_atoms, H2_coords, charge=H2_charge, multiplicity=H2_multiplicity)

다음으로 molecular variation의 type을 다음과 같이 Molecule.absolute_stretching으로 지정한다.

In [ ]:
molecular_variation = Molecule.absolute_stretching

atom_pair는 첫번째 원자를 두번쨰 원자에 대해 거리를 조절하는 지를 지정한다. 숫자는 위의 atoms의 list에서 원자의 index를 말한다. 만약 LiH의 경우 atoms = ["Li","H"]이고 atom_pair는 (1,0)이면 첫번째 원자인 index 1 즉, atoms[1]의 원자 H가 두번째 원자 index 2 즉, atoms[0]의 원자 Li에 대하여 거리를 조절하겠다는 의미이다.

In [ ]:
specific_molecular_variation = apply_variation(molecular_variation, atom_pair=(1, 0))

위의 변수들로 초기 분자 정의를 바꾼다. 위의 과정으로 atom_pair를 이용하여 degrees of freedom을 얻은 후 degrees_of_freedom을 변수로 넣음.

In [ ]:
# The MoleculeInfo has become a pure data container and no longer supports degrees of freedom.
H2_molecule_stretchable = Molecule(geometry=
                                 [['H', [0., 0., 0.]], ['H', [0., 0., 0.735]]], charge=0, multiplicity=1,
                                 degrees_of_freedom=[specific_molecular_variation])

다음으론 주어진 분자의 정보들로 solve_ground_state()를 실행하여 VQE를 사용한다. 그리고 H2_fermionic_hamiltonian, H2_num_particles, H2_num_spin_orbitals, H2_qubit_op, H2_qubit_converter, H2_ground_state를 얻는다. 다음 코드는 NumPy exact minimum eignesolver를 사용할 경우이다.

In [ ]:
H2_fermionic_hamiltonian, H2_num_particles, H2_num_spin_orbitals, H2_qubit_op, H2_qubit_converter, H2_ground_state = \
                  solve_ground_state(H2_info_dict, mapper_name="Parity",
                  two_qubit_reduction=True, z2symmetry_reduction=None, 
                  name_solver = 'NumPy exact solver', solver = numpy_solver)

다음 코드는 UCCSD를 사용하여 VQE를 실행했을 때의 코드와 결과이다.

In [ ]:
H2_fermionic_hamiltonian, H2_num_particles, H2_num_spin_orbitals, H2_qubit_op, H2_qubit_converter, H2_ground_state = \
                  solve_ground_state(H2_info_dict, mapper_name="Parity",
                  two_qubit_reduction=True, z2symmetry_reduction=None, 
                  name_solver = 'UCCSD ansatz')

다음 코드는 heuristic ansatz와 SLSQP optimizer와 함께 Two-Local circuit을 사용하여 VQE를 실행한 코드와 결과이다.

In [ ]:
H2_fermionic_hamiltonian, H2_num_particles, H2_num_spin_orbitals, H2_qubit_op, H2_qubit_converter, H2_ground_state = \
                  solve_ground_state(H2_info_dict, mapper_name="Parity",
                   two_qubit_reduction=True, z2symmetry_reduction=None, 
                   name_solver = 'Heuristic ansatz, the Two-Local circuit with SLSQP',solver = vqe_tl_solver)

다음으론 heuristic ansatz와 SLSQP optimizer와 함께 Two-Local circuit을 사용하여 VQE를 실행하여 loss function의 plot을 그려보자.

먼저 qnspsa_loss를 다음과 같이 정의한다.

In [ ]:
#https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.QNSPSA.html?highlight=qnspsa#qiskit.algorithms.optimizers.QNSPSA
#qiskit.algorithms.optimizers.QNSPSA.get_fidelity
#https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.QNSPSA.get_fidelity.html
qnspsa_loss = []
ansatz = tl_circuit
#fidelity = QNSPSA.get_fidelity(ansatz, quantum_instance, expectation=PauliExpectation())
fidelity = QNSPSA.get_fidelity(ansatz, sampler=Sampler())
qnspsa = QNSPSA(fidelity, maxiter=200, learning_rate=0.01, perturbation=0.7, callback=qnspsa_callback)

다음으론 위 코드에서 선언한 변수로 VQE를 setting한다.

In [ ]:
vqe_tl_QNSPSA_solver = VQE(estimator, tl_circuit, optimizer=qnspsa)

그리고 solve_ground_state()를 사용하여 loss function의 plot을 그린다.이때 heuristic ansatz와 QN-SPSA optimizer를 쓴다.

In [ ]:
H2_fermionic_hamiltonian, H2_num_particles, H2_num_spin_orbitals, H2_qubit_op, H2_qubit_converter, H2_ground_state = \
                  solve_ground_state(H2_info_dict, mapper_name="Parity", two_qubit_reduction=True, z2symmetry_reduction=None,
                  loss=qnspsa_loss, label='QN-SPSA', target=-1.857274810366,
                  name_solver='Two-Local circuit and the QN-SPSA optimizer', solver=vqe_tl_QNSPSA_solver)

다음 그래프는 다양한 방법을 사용했을 때 얻은 결과이다. 

![figure_5.11.png](attachment:./img/figure_5.11.png)

위의 표는 같은 mapper (ParityMapper())와 two_qubit_reduction=True일때  electronic ground state와 전체 ground state energy가 얼마나 다른지를 비교하는 표이다. PyQMC method가 가장 낮은 전체 에너지 -1.162Ha 를 얻고 이 값이 가장 정확한 값이다. 

#### Computing the BOPES

다음으론 수소 분자의 BOPES을 계산하고 그래프로 그리는 코드이다.

In [ ]:
# BOPES of H2
perturbation_steps = np.linspace(-0.5, 2, 25) # 25 equally spaced points from -0.2 to 4, inclusive.
H2_s_fermionic_hamiltonian, H2_s_num_particles, H2_s_num_spin_orbitals, H2_s_qubit_op, H2_s_qubit_converter, H2_s_ground_state = \
                  solve_ground_state(H2_info_dict, mapper_name="Parity",
                   two_qubit_reduction=True, z2symmetry_reduction=None, 
                   name_solver = 'NumPy exact solver', solver = numpy_solver,
                   plot_bopes = True, perturbation_steps=perturbation_steps)

### 5.2.2. Lithium hydride molecule

이 section은 LiH에 적용하는 과정이다. H2와 크게 다르지 않다.
먼저 LiH의 기본 정보를 입력하는 과정이다.

In [ ]:
# LiH molecule
LiH_atoms = ["Li", "H"]
LiH_coords = [(0.0, 0.0, 0.0), (0.0, 0.0, 1.5474)]
LiH_charge = 0
LiH_multiplicity = 1
LiH_atom_pair=(1,0)
LiH_info_dict={'atoms':LiH_atoms, 'coords':LiH_coords, 'charge':LiH_charge, 'multiplicity':LiH_multiplicity, 'atom_pair':LiH_atom_pair}

LiH_moleculeinfo = MoleculeInfo(LiH_atoms, LiH_coords, charge=LiH_charge, multiplicity=LiH_multiplicity)

### Varying the lithium hydride molecule

In [ ]:
# Vary LiH
LiH_molecule_stretchable = Molecule(geometry=[['Li', [0., 0., 0.]], ['H', [0., 0., 1.5474]]], charge=0, multiplicity=1,
                                   degrees_of_freedom=[specific_molecular_variation])

### Solving for the Ground-state

다음 코드는 Numpy exact solver를 사용하여 VQE로 ground state를 얻는 과정이다.

In [ ]:
# Ground state for LiH

LiH_fermionic_hamiltonian, LiH_num_particles, LiH_num_spin_orbitals, LiH_qubit_op, LiH_qubit_converter, LiH_ground_state = \
                  solve_ground_state(LiH_info_dict, mapper_name="Parity",
                  freeze_core=FreezeCoreTransformer(freeze_core=True, remove_orbitals=[4, 3]),
                  two_qubit_reduction=True, z2symmetry_reduction="auto", 
                  name_solver='NumPy exact solver', solver=numpy_solver)

다음은 SLSQP와 Two-Local circuit을 사용한 VQE를 사용한 코드이다.

In [ ]:
LiH_fermionic_hamiltonian, LiH_num_particles, LiH_num_spin_orbitals, LiH_qubit_op, LiH_qubit_converter, LiH_ground_state = \
                  solve_ground_state(LiH_info_dict, mapper_name="Parity",
                  freeze_core=FreezeCoreTransformer(freeze_core=True, remove_orbitals=[4, 3]),
                  two_qubit_reduction=True, z2symmetry_reduction="auto", 
                  name_solver = 'Heuristic ansatz, the Two-Local circuit with SLSQP', solver = vqe_tl_solver)

다음은 위의 H2의 경우와 마찬가지로 heuristic ansatz와 SLSQP optimizer와 함께 Two-Local circuit을 사용하여 VQE를 실행하여 loss function의 plot을 그리는 과정이다.

In [ ]:
qnspsa_loss = []
ansatz = tl_circuit
#fidelity = QNSPSA.get_fidelity(ansatz, quantum_instance, expectation=PauliExpectation())
fidelity = QNSPSA.get_fidelity(ansatz, sampler=Sampler())
qnspsa = QNSPSA(fidelity, maxiter=500, learning_rate=0.01, perturbation=0.7, callback=qnspsa_callback)

In [ ]:
vqe_tl_QNSPSA_solver = VQE(estimator, tl_circuit, optimizer=qnspsa)

In [ ]:
LiH_fermionic_hamiltonian, LiH_num_particles, LiH_num_spin_orbitals, LiH_qubit_op, LiH_qubit_converter, LiH_ground_state = \
                  solve_ground_state(LiH_info_dict, mapper_name="Parity",
                  freeze_core=FreezeCoreTransformer(freeze_core=True, remove_orbitals=[4, 3]),
                  two_qubit_reduction=True, z2symmetry_reduction="auto", loss=qnspsa_loss, label='QN-SPSA', target=-1.0703584,
                  name_solver='Two-Local circuit and the QN-SPSA optimizer', solver=vqe_tl_QNSPSA_solver)

다음 표는 H2의 경우와 같이 여러가지 solver와 다양한 package를 사용하여 얻은 결과의 비교표이다.

![figure_5.18.png](attachment:figure_5.18.png)

위의 표에 결과도 역시 PyQMC가 가장 낮고 가장 정확한 에너지 -8.102Ha 를 갖는다.

#### Computing the Born-Oppenheimer Potential Energy Surface (BOPES)
마지막으로 LiH의 BOPES를 계산하고 그래프로 그리는 코드이다.

In [ ]:
# BOPES LiH

perturbation_steps = np.linspace(-0.8, 0.8, 10) # 10 equally spaced points from -0.8 to 0.8, inclusive.
LiH_s_fermionic_hamiltonian, LiH_s_num_particles, LiH_s_num_spin_orbitals, LiH_s_qubit_op, LiH_s_qubit_converter, LiH_s_ground_state = \
                  solve_ground_state(LiH_info_dict, mapper_name="Parity",
                  freeze_core=FreezeCoreTransformer(freeze_core=True, remove_orbitals=[4, 3]),
                   two_qubit_reduction=True, z2symmetry_reduction="auto", 
                   name_solver='NumPy exact solver', solver=numpy_solver,
                   plot_bopes=True, perturbation_steps=perturbation_steps)

### 5.2.3. Marcro molecule

이번 section에선 IBM Quantum Challenge Africa 2021, Quantum Chemistry for HIV의 case를 사용하여 HIV를 볼 것이다. Challenge에선 안티 레트로 바이러스(anti-retroviral) 분자의 toy model이 단백질 분해 효소(protease) 분자와 결합될 수 있는지를 얻는데 목적을 두고 있다. 안티 레트로 바이러스 분자는 많은 원자를 갖고 있기에 단일 탄소 분자로 근사했다. 그리고 단백질 분해 효소의 toy model은 포름아미드(formamide) 분자 - HCONH2로 표현하였다. 여기에선 특별히 포름아미드 분자의 탄소-산소-질소 부분으로 간단히 말해서 이 코드는 단일 탄소 원자가 포름아미드 분자의 탄소-산소-질소 부분이 결합될 수 있는지를 보는 과정이다.

#### ASE atomic visulization

먼저 macro molecule을 정의한다. 이때 ASE의 Atoms를 사용한다.

In [ ]:
# Macro Molecule

macro_ASE = Atoms('ONCHHHC', [(1.1280, 0.2091, 0.0000), 
                          (-1.1878, 0.1791, 0.0000), 
                          (0.0598, -0.3882, 0.0000),
                          (-1.3085, 1.1864, 0.0001),
                          (-2.0305, -0.3861, -0.0001),
                          (-0.0014, -1.4883, -0.0001),
                          (-0.1805, 1.3955, 0.0000)])

다음으론 위의 정의한 분자를 ASE viewer X3D를 사용하여 분자를 3D로 보는 코드이다.

In [ ]:
view(macro_ASE, viewer='x3d')

다음 과정은 위의 H2와 LiH의 과정과 비슷한 과정들이다.
molecular_variation을 Molecule.absolute_stretching으로 얻고 specific_molecular_variation을 얻는다.

In [ ]:
molecular_variation = Molecule.absolute_stretching

In [ ]:
specific_molecular_variation = apply_variation(molecular_variation, atom_pair=(6, 1))

다음은 macro 분자의 정보를 입력하는 과정이다. 분자가 가진 원자, 각 원자의 위치, charge, multiplicity, atom pair를 입력하고 원하는 자료구조로 저장한다. 현재 문제 상황은 HCONH2에 C를 결합하는 것이므로 각 원자의 좌표를 입력하고 HCONH2의 N에 C가 가까워지는 상황을 만들었다. 이것은 아래 코드에 atom_pair에 입력되어 있다.

In [ ]:
M_atoms = ["O", "N","C","H","H","H","C"]
M_coords = [(1.1280, 0.2091, 0.0000), 
                    (-1.1878, 0.1791, 0.0000),
                    (0.0598, -0.3882, 0.0000),
                    (-1.3085, 1.1864, 0.0001),
                    (-2.0305, -0.3861, -0.0001),
                    (-0.0014, -1.4883, -0.0001),
                    (-0.1805, 1.3955, 0.0000)]
M_charge = 0
M_multiplicity = 1
M_atom_pair=(6, 1)
M_info_dict={'atoms':M_atoms, 'coords':M_coords, 'charge':M_charge, 'multiplicity':M_multiplicity, 'atom_pair':M_atom_pair}

macromoleculeinfo = MoleculeInfo(M_atoms, M_coords, charge=M_charge, multiplicity=M_multiplicity)

### Solving for the Ground-state

다음으론 주어진 정보로 VQE로 ground state를 얻는 과정이다. 사용한 condition은 다음과 같다.

In [ ]:
#Ground State for Macro Molecule
print("Macro molecule")
print("Using the ParityMapper with two_qubit_reduction=True to eliminate two qubits")
print("Parameters ActiveSpaceTransformer(num_electrons=2, num_molecular_orbitals=2)")
print("Setting z2symmetry_reduction=\"auto\"")

In [ ]:
macro_fermionic_hamiltonian, macro_particle_num_particles, macro_particle_num_spin_orbitals, macro_qubit_op, macro_qubit_converter, macro_ground_state = \
                  solve_ground_state(M_info_dict, mapper_name="Parity",
                  num_electrons=2, num_spatial_orbitals=2,
                  two_qubit_reduction=True, z2symmetry_reduction="auto", 
                  name_solver='NumPy exact solver', solver=numpy_solver, pyqmc=False)

### Computing the BOPES

마지막으로 macro 분자의 BOPES를 계산하고 그래프로 그리는 코드이다.

In [ ]:
# BOPES of Macro Molecule
perturbation_steps = np.linspace(-0.5, 3, 20) # 20 equally spaced points from -0.5 to 3, inclusive.
macro_fermionic_hamiltonian, macro_particle_num_particles, macro_particle_num_spin_orbitals, macro_qubit_op, macro_qubit_converter, macro_ground_state = \
                  solve_ground_state(M_info_dict, mapper_name="Parity",
                  num_electrons=2, num_spatial_orbitals=2,
                  two_qubit_reduction=True, z2symmetry_reduction="auto",  
                  name_solver='NumPy exact solver', solver=numpy_solver, pyqmc=False,
                  plot_bopes=True, perturbation_steps=perturbation_steps)

위의 결과에서 macro 분자의 POPES의 그래프는 정확한 최솟값을 나타내지 못한다. 따라서 단일 탄소 원자는 포름아미드의 단백질 분해효소 모형에 결합되지 않는다고 결론 내릴 수 있다.

## Summary